<a href="https://colab.research.google.com/github/RemyLpr/defi_ia/blob/main/feature_engineering/stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dans ce notebook on va rassembler toutes nos données dans le but d'obtenir le stock de chaque hôtel à chaque date. Il y aura des données manquantes que nous allons inférer dans un second temps.

# Import des librairies et des données

In [85]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import normalize
from scipy.spatial import distance
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_vienna = pd.read_csv("vienna.csv")
df_vilnius = pd.read_csv("vilnius.csv")
df_amsterdam = pd.read_csv("amsterdam.csv")
df_rome = pd.read_csv("rome.csv")
df_valletta = pd.read_csv("valletta.csv")
df_paris = pd.read_csv("paris.csv")
df_madrid = pd.read_csv("madrid.csv")
df_copenhagen = pd.read_csv("copenhagen.csv")
df_sofia = pd.read_csv("sofia.csv")
df = pd.concat([df_vienna, df_vilnius, df_amsterdam, df_rome, df_valletta, df_paris, df_madrid, df_copenhagen, df_sofia])
df_test = pd.read_csv("test_set.csv")

# Chargement des stocks issus des requêtes et du test set

In [6]:
stock_list = []
for hotel in range(0, 999):
  df_hot = df[df["hotel_id"] == hotel]
  df_hot_test = df_test[df_test["hotel_id"] == hotel]
  stock_per_hotel = []
  for date in range(0, 45):
    df_hot_date = df_hot[df_hot["date"] == date]
    df_hot_test_date = df_hot_test[df_hot_test["date"] == date]
    if len(df_hot_date) > 0:
      stock = df_hot_date["stock"].mean() # moyenne inutile ici, c'est juste pour transformer en valeur
    else:
      stock = df_hot_test_date["stock"].mean() # cas NaN compris ici
    stock_per_hotel.append(stock)
  stock_list.append(stock_per_hotel)

In [117]:
df_stock = pd.DataFrame(data = stock_list)
df_na = df_stock[df_stock.isna().any(axis=1)]
df_no_na = df_stock.dropna()

# Inférence des valeurs manquantes

In [ ]:
for index, row in df_na.iterrows():
  rowcopy = row.copy()
  list_index = []
  list_dist = []
  list_na = []
  list_row = []
  newrow = []
  # on connaît les jours pour lesquels les stocks sont toujours manquants pour ces hôtels
  list_na = [7, 8, 9, 10, 11, 12, 13, 14, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
  for i in sorted(list_na, reverse=True):
      del row[i]
  normalized_row = normalize([row])  
  for index2, row2 in df_no_na.iterrows():
    for i in sorted(list_na, reverse=True):
      del row2[i]
    normalized_row2 = normalize([row2])
    dist = distance.euclidean(normalized_row, normalized_row2)
    list_index.append(index2)
    list_dist.append(dist)
    list_row.append(row2)
  indice_min = np.argmin(list_dist)
  voisin = df_no_na.iloc[indice_min]

  # pour afficher les paires hotel de df_na et son + proche voisin
  #print(index, voisin.name)

  # calcul du coeff entre les 2 hôtels voisins
  bg_A_1, bd_A_1, bg_A_2, bd_A_2 = row[6], row[15], row[21], row[34] # pour l'hôtel à stocks manquants
  bg_B_1, bd_B_1, bg_B_2, bd_B_2 = row2[6], row2[15], row2[21], row2[34] # pour le voisin
  x = row.values.tolist()
  y = list_row[indice_min].values.tolist()
  rapport_list = []
  for i in range(0, len(x)):
    if x[i] != 0 and y[i] != 0: 
      rapport_list.append(x[i] / y[i])
  coeff = round(np.mean(rapport_list), 2)

  # remplissage des trous dans les stocks
  for j in range(len(rowcopy)):
    if j in [7, 8, 9, 10, 11, 12, 13, 14]: # premier trou
      if bg_B_1 == bd_B_1: # cas borne gauche = borne droite, on remplit par cette valeur
        newrow.append(bg_A_1)
      else:
        estimation = round(voisin[j]*coeff)
        if (estimation < bg_A_1) & (bg_A_1 !=0): # cas estimation < borne gauche, absurde car le stock croît avec la variable date
          newrow.append(bg_A_1)
        elif estimation > bd_A_1: # cas estimation > borne droite, absurde pour la même raison
          newrow.append(bd_A_1)
        else:
          newrow.append(round(voisin[j]*coeff))
    elif j in [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]: # second trou
      if bg_B_2 == bd_B_2: # cas borne gauche = borne droite, on remplit par cette valeur
        newrow.append(bg_A_2)
      else:
        estimation = round(voisin[j]*coeff)
        if (estimation < bg_A_2) & (bg_A_2 !=0): 
          newrow.append(bg_A_2)
        elif estimation > bd_A_2: 
          newrow.append(bd_A_2)
        else:
          newrow.append(estimation)
    else:
      newrow.append(rowcopy[j])
  df_na.at[index] = newrow

# Export des résultats

In [116]:
df_stock = pd.concat([df_na, df_no_na])
df_stock.to_csv("stock.csv", index = False)